(pymc3_schema)=
# Example of `InferenceData` schema in PyMC3

The description of the `InferenceData` structure can be found {ref}`here <schema>`.

In [1]:
import arviz as az
import pymc3 as pm
import pandas as pd
import numpy as np
import xarray
xarray.set_options(display_style="html");

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time = data.time.values
slack_comments = data.comments.values
github_commits = data.commits.values
names = data.index.values
N = len(names)
data

,comments,commits,time
Alice,7500,25,4.5
Bob,10100,32,6.0
Cole,18600,49,7.0
Danielle,25200,66,12.0
Erika,27500,96,18.0


In [3]:
# data for out of sample predictions
candidate_devs = ["Francis", "Gerard"]
candidate_devs_time = np.array([3.6, 5.1])

In [4]:
dims={
    "slack_comments": ["developer"],
    "github_commits": ["developer"],
    "time_since_joined": ["developer"],
}
with pm.Model() as model:
    time_since_joined = pm.Data("time_since_joined", time)
    
    b_sigma = pm.HalfNormal('b_sigma', sd=300)
    c_sigma = pm.HalfNormal('c_sigma', sd=6)
    b0 = pm.Normal("b0", mu=0, sd=200)
    b1 = pm.Normal("b1", mu=0, sd=200)
    c0 = pm.Normal("c0", mu=0, sd=10)
    c1 = pm.Normal("c1", mu=0, sd=10)
    
    pm.Normal("slack_comments", mu=b0 + b1 * time_since_joined, sigma=b_sigma, observed=slack_comments)
    pm.Normal("github_commits", mu=c0 + c1 * time_since_joined, sigma=c_sigma, observed=github_commits)
    
    trace = pm.sample(400, chains=4)
    posterior_predictive = pm.sample_posterior_predictive(trace)
    prior = pm.sample_prior_predictive(150)
    idata_pymc3 = az.from_pymc3(
        trace,
        prior=prior,
        posterior_predictive=posterior_predictive,
        coords={"developer": names},
        dims=dims
    )

Only 400 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [c1, c0, b1, b0, c_sigma, b_sigma]


There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [5]:
dims_pred={
    "slack_comments": ["candidate developer"],
    "github_commits": ["candidate developer"],
    "time_since_joined": ["candidate developer"],
}
with model:
    pm.set_data({"time_since_joined": candidate_devs_time})
    predictions = pm.sample_posterior_predictive(trace)
    az.from_pymc3_predictions(
        predictions, 
        idata_orig=idata_pymc3, 
        inplace=True,
        coords={"candidate developer": candidate_devs},
        dims=dims_pred,
    )

In [6]:
idata_pymc3

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats
	> prior
	> prior_predictive
	> observed_data
	> log_likelihood
	> constant_data
	> predictions
	> predictions_constant_data

In this example, each variable has as dimensions a combination of the following three: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values.

* In the case of `chain` and `draw` it is an integer identifier starting at `0`.
* In the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [7]:
idata_pymc3.posterior

<xarray.Dataset>
Dimensions:  (chain: 4, draw: 400)
Coordinates:
  * chain    (chain) int64 0 1 2 3
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
Data variables:
    b0       (chain, draw) float64 619.3 313.6 88.4 236.1 ... 325.9 273.1 185.8
    b1       (chain, draw) float64 1.516e+03 1.525e+03 ... 1.621e+03 1.485e+03
    c0       (chain, draw) float64 -15.82 -10.81 -15.5 ... 6.432 4.305 2.761
    c1       (chain, draw) float64 6.507 7.465 6.517 7.087 ... 5.168 5.07 5.189
    b_sigma  (chain, draw) float64 1.512e+03 1.815e+03 ... 1.412e+03 1.8e+03
    c_sigma  (chain, draw) float64 7.493 15.52 8.19 13.64 ... 4.684 4.379 5.154
Attributes:
    created_at:                 2020-01-27T22:36:59.510665
    inference_library:          pymc3
    inference_library_version:  3.8

In [8]:
idata_pymc3.sample_stats

<xarray.Dataset>
Dimensions:           (chain: 4, draw: 400)
Coordinates:
  * chain             (chain) int64 0 1 2 3
  * draw              (draw) int64 0 1 2 3 4 5 6 ... 393 394 395 396 397 398 399
Data variables:
    energy_error      (chain, draw) float64 0.1398 -0.232 ... 0.3555 -0.02345
    energy            (chain, draw) float64 149.4 151.4 150.4 ... 142.6 140.7
    step_size_bar     (chain, draw) float64 0.3863 0.3863 ... 0.4529 0.4529
    tree_size         (chain, draw) float64 15.0 15.0 7.0 7.0 ... 7.0 7.0 15.0
    max_energy_error  (chain, draw) float64 -0.1656 -0.6437 ... 1.534 -0.06507
    diverging         (chain, draw) bool False False False ... False False False
    lp                (chain, draw) float64 -148.2 -147.3 ... -139.5 -139.2
    depth             (chain, draw) int64 4 4 3 3 3 3 3 3 3 ... 4 4 4 3 4 3 3 4
    tune              (chain, draw) bool True False False ... False False False
    mean_tree_accept  (chain, draw) float64 0.9897 0.9751 ... 0.713 0.9966
    step_size         (chain, draw) float64 0.3952 0.3952 ... 0.4748 0.4748
Attributes:
    created_at:                 2020-01-27T22:36:59.515483
    inference_library:          pymc3
    inference_library_version:  3.8

In [9]:
idata_pymc3.log_likelihood

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 400)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 393 394 395 396 397 398 399
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 -8.241 -8.273 ... -8.468
    github_commits  (chain, draw, developer) float64 -4.118 -3.619 ... -2.559
Attributes:
    created_at:                 2020-01-27T22:37:00.122554
    inference_library:          pymc3
    inference_library_version:  3.8

In [10]:
idata_pymc3.posterior_predictive

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 400)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 ... 393 394 395 396 397 398 399
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (chain, draw, developer) float64 6.758e+03 ... 2.689e+04
    github_commits  (chain, draw, developer) float64 2.162 21.91 ... 70.43 88.62
Attributes:
    created_at:                 2020-01-27T22:37:00.125917
    inference_library:          pymc3
    inference_library_version:  3.8

In [11]:
idata_pymc3.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) float64 7.5e+03 1.01e+04 ... 2.52e+04 2.75e+04
    github_commits  (developer) float64 25.0 32.0 49.0 66.0 96.0
Attributes:
    created_at:                 2020-01-27T22:37:00.136543
    inference_library:          pymc3
    inference_library_version:  3.8

In [12]:
idata_pymc3.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2020-01-27T22:37:00.137808
    inference_library:          pymc3
    inference_library_version:  3.8

In [13]:
idata_pymc3.prior

<xarray.Dataset>
Dimensions:  (chain: 1, draw: 150)
Coordinates:
  * chain    (chain) int64 0
  * draw     (draw) int64 0 1 2 3 4 5 6 7 8 ... 142 143 144 145 146 147 148 149
Data variables:
    b0       (chain, draw) float64 -93.49 -417.1 -159.3 ... -386.8 261.1 -114.7
    b1       (chain, draw) float64 166.7 391.6 -41.08 ... -19.23 15.14 120.7
    c0       (chain, draw) float64 2.135 8.599 2.809 ... 15.09 5.763 -7.103
    c1       (chain, draw) float64 2.985 -3.813 -13.98 ... -2.95 -13.68 10.66
    b_sigma  (chain, draw) float64 528.2 726.8 438.8 185.6 ... 145.8 36.9 127.3
    c_sigma  (chain, draw) float64 1.223 3.522 0.4642 ... 2.649 11.57 3.553
Attributes:
    created_at:                 2020-01-27T22:37:00.129095
    inference_library:          pymc3
    inference_library_version:  3.8

In [14]:
idata_pymc3.predictions

<xarray.Dataset>
Dimensions:              (candidate developer: 2, chain: 1, draw: 1600)
Coordinates:
  * chain                (chain) int64 0
  * draw                 (draw) int64 0 1 2 3 4 5 ... 1595 1596 1597 1598 1599
  * candidate developer  (candidate developer) <U7 'Francis' 'Gerard'
Data variables:
    slack_comments       (chain, draw, candidate developer) float64 3.537e+03 ... 8.227e+03
    github_commits       (chain, draw, candidate developer) float64 4.627 ... 35.58
Attributes:
    created_at:                 2020-01-27T22:37:07.840756
    inference_library:          pymc3
    inference_library_version:  3.8

In [15]:
idata_pymc3.predictions_constant_data

<xarray.Dataset>
Dimensions:              (candidate developer: 2)
Coordinates:
  * candidate developer  (candidate developer) <U7 'Francis' 'Gerard'
Data variables:
    time_since_joined    (candidate developer) float64 3.6 5.1
Attributes:
    created_at:                 2020-01-27T22:37:07.842636
    inference_library:          pymc3
    inference_library_version:  3.8